In [1]:
import pickle

In [8]:
file = open('../raw_data/audio_features_first28750.txt',"rb")
audioFile = pickle.load(file)

In [7]:
from collections import defaultdict
import pandas as pd

In [9]:
audioFile

[{'danceability': 0.567,
  'energy': 0.198,
  'key': 4,
  'loudness': -18.681,
  'mode': 1,
  'speechiness': 0.0327,
  'acousticness': 0.919,
  'instrumentalness': 0.000871,
  'liveness': 0.135,
  'valence': 0.321,
  'tempo': 92.467,
  'type': 'audio_features',
  'id': '7ny2ATvjtKszCpLpfsGnVQ',
  'uri': 'spotify:track:7ny2ATvjtKszCpLpfsGnVQ',
  'track_href': 'https://api.spotify.com/v1/tracks/7ny2ATvjtKszCpLpfsGnVQ',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7ny2ATvjtKszCpLpfsGnVQ',
  'duration_ms': 412200,
  'time_signature': 4},
 {'danceability': 0.579,
  'energy': 0.479,
  'key': 5,
  'loudness': -10.348,
  'mode': 1,
  'speechiness': 0.0294,
  'acousticness': 0.206,
  'instrumentalness': 0.00211,
  'liveness': 0.163,
  'valence': 0.937,
  'tempo': 135.459,
  'type': 'audio_features',
  'id': '0wfBooQ6QtDm63UIyIcDIj',
  'uri': 'spotify:track:0wfBooQ6QtDm63UIyIcDIj',
  'track_href': 'https://api.spotify.com/v1/tracks/0wfBooQ6QtDm63UIyIcDIj',
  'analysis_url': 'http

In [13]:
extract = defaultdict(list)
for file in audioFile:
    for key in file:
        extract[str(key)].append(file[key])

In [14]:
dfAudio = pd.DataFrame.from_dict(extract)

In [16]:
dfAudio = dfAudio.rename(columns = {'uri': 'track_uri'})

In [18]:
dfAudio.shape

(28850, 18)

In [19]:
dfAudio_unique = dfAudio.drop_duplicates()

In [20]:
dfAudio_unique.shape

(26253, 18)

In [30]:
dfAudio_unique.to_csv('../raw_data/audio_features_first28750.csv')

In [26]:
dfAudio_unique.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'track_uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [28]:
import sqlite3
import csv
import numpy as np

In [25]:
#connect to database
db = sqlite3.connect('../raw_data/spotify.db')
cursor = db.cursor()

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [29]:
#prep new table
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
#create a new table for features_partial
cursor.execute('''CREATE TABLE IF NOT EXISTS audio_features_first28750
               (Track_uri TEXT, duration_ms, acousticness REAL, danceability, 
                energy, explicit,instrumentalness, key, liveness, loudness, mode, 
                speechiness, tempo, valence, time_signature)''')

In [32]:
#write information from audio_features_first28750.csv into database
with open('../raw_data/audio_features_first28750.csv') as file:
    info = csv.reader(file, delimiter = ',')
    next(info, None)
    for row in info:
        index, danceability, energy, key, loudness, mode, speechiness, acousticness, intrumentalness, liveness, valence, tempo, atype, aid, track_uri, track_href, analysis_url, duration_ms, time_signature = row
        insert_into_db = (track_uri, duration_ms, acousticness, danceability, energy, 0, intrumentalness, key, liveness, loudness, mode, speechiness, tempo, valence, time_signature)
        cursor.execute('''INSERT INTO audio_features_first28750
                         (Track_uri, duration_ms, acousticness, danceability, 
                          energy, explicit,instrumentalness, key, liveness, loudness, mode, 
                          speechiness, tempo, valence, time_signature) 
                          VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', insert_into_db)

In [33]:
audio_cols = [col[1] for col in cursor.execute("PRAGMA table_info(audio_features_first28750)")]

In [34]:
query = '''SELECT * FROM audio_features_first28750 LIMIT 5'''
viz_tables(audio_cols, query)

,Track_uri,duration_ms,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,time_signature
0,spotify:track:7ny2ATvjtKszCpLpfsGnVQ,412200,0.919,0.567,0.198,0,0.000871,4,0.135,-18.681,1,0.0327,92.467,0.321,4
1,spotify:track:0wfBooQ6QtDm63UIyIcDIj,177200,0.206,0.579,0.479,0,0.00211,5,0.163,-10.348,1,0.0294,135.459,0.937,4
2,spotify:track:3xNO6JUk9ySYjxuqzcZYRB,288000,0.768,0.383,0.224,0,0.0,2,0.087,-16.99,1,0.0274,102.94,0.466,4
3,spotify:track:5ka7NFOlZUpVLJmA2tO0o4,389800,0.565,0.666,0.617,0,0.0,7,0.0498,-10.851,1,0.0628,105.648,0.764,4
4,spotify:track:2FDTgrYDUdQKyGmtynEk90,171107,0.571,0.702,0.56,0,0.0,4,0.109,-12.323,1,0.0342,125.452,0.889,4


In [35]:
db.commit()
db.close()